# Scrape data
## Install dependencies

In [61]:
from selectolax.parser import HTMLParser
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time

## retrives hotels' detail page

In [67]:
# page url to collect links to hotel's detail
hotel_list_url = "https://www.booking.com/searchresults.html?aid=304142&label=gen173nr-1FCAMo9AE4qANIM1gEaPQBiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuALa4LSyBsACAdICJGM5MzUxNDk2LWYyZjItNDdlNC1iOTkzLTQ0OWQ2YTQzMTEzNtgCBeACAQ&city=-3730078"
page_source = None

chrome_options = webdriver.ChromeOptions()
# Add any additional options here if needed

def launch_page(driver, url):
    driver.get(url)
    # Wait for dismiss button to appear on screen
    time.sleep(5)
    # Find the button "Dismiss sign-in info."
    # Click the button 
    dismiss_button = driver.find_element(By.XPATH, "//button[@aria-label='Dismiss sign-in info.']")
    dismiss_button.click()
    time.sleep(1)

    # Locate the div with the specific data-testid attribute
    searchbox_dates_container = driver.find_element(By.CSS_SELECTOR, 'div[data-testid="searchbox-dates-container"]')
    searchbox_dates_container.click()

    # Locate the span with the specific aria-label attribute
    date_span = driver.find_element(By.CSS_SELECTOR, 'div[data-testid="searchbox-layout-wide"] span[aria-label="22 May 2024"]')
    date_span.click()

    # Locate the span with the specific aria-label attribute
    date_span = driver.find_element(By.CSS_SELECTOR, 'div[data-testid="searchbox-layout-wide"] span[aria-label="23 May 2024"]')
    date_span.click()

    # Locate the search with the specific aria-label attribute
    search_button = driver.find_element(By.CSS_SELECTOR, 'button[type="submit"]')
    search_button.click()

    time.sleep(4)  # Wait for loading more items

    # Scroll to bottom to load more item
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight-500);")
    time.sleep(5)  # Wait for loading more items
    # Load more button should be displayed at this point

    # Load more result when by clicking Load more button 
    for _ in range(1, 5):
        # Find the button by its aria-label "Dismiss sign-in info."
        # Click the button 
        load_more_button = driver.find_element(By.XPATH, "//span[contains(text(), 'Load more results')]/parent::button")
        load_more_button.click()
        time.sleep(3)

    # Close the browser
    # driver.quit()
    return driver.page_source

# Initialize the WebDriver using webdriver-manager
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# Launch the page
page_source = launch_page(driver, hotel_list_url)

driver.quit()

page_html = HTMLParser(page_source)

# Find the div with data-testid="property-card-container"
links = []

# Extract href attribute from each link
for link in page_html.css('div[data-testid="property-card-container"] h3 a'):
    href = link.attributes['href']
    links += [href]

print("number of detail pages: ", len(links))



number of detail pages:  175


## Extract data from detail page

In [70]:
service = Service(ChromeDriverManager().install())
# chrome_options.add_argument("--headless")  # Run in headless mode

# parse element's content to text, remove extra endline and whitespace
def parse_text(html, selector):
    return html.css_first(selector).text().strip() if html.css(selector) else None


rooms = []

for page in range(0, len(links)): 
    page_html = None
    # Launch the page
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.get(links[page])
    time.sleep(1)

    try:
        page_html = HTMLParser(driver.page_source)
    finally:
        # Quit the WebDriver
        driver.quit()

    for row in page_html.css(".hprt-table tbody tr"): 
        room ={
            "hotel_name": parse_text(page_html, '.pp-header__title'),
            "hotel_star": len(page_html.css('[data-testid="rating-stars"] span')),
            "room_name": parse_text(row, '.hprt-roomtype-icon-link'),
            "room_area": parse_text(row, '.hprt-table-cell [data-name-en="room size"] span'),
            "bed_detail": ",".join([parse_text(bed,"span") for bed in row.css(".rt-bed-type span")]),
            "available_guests": len(row.css(".c-occupancy-icons__adults i")),
            "root_price": parse_text(row, '.prco-inline-block-maker-helper'),
            "discount_rate": parse_text(row, '.bui-badge__text'),
            "price":  parse_text(row, '.hprt-table-cell-price .prco-valign-middle-helper'),
            "description": parse_text(row, '.hprt-block p.short-room-desc')
        }
        rooms.append(room)

    print("Scrape page {} success!".format(parse_text(page_html, '.pp-header__title')))
    print("Number of item scraped: {} items.".format(len(rooms)))





Scrape page Ruby Saigon Hotel Le Thanh Ton success!
Number of item scraped: 4 items.
Scrape page Elysium CBD Rivergate Apartment - Gym & Pool - Free 4G sim for 3 nights success!
Number of item scraped: 11 items.
Scrape page HANZ SEN Hotel success!
Number of item scraped: 13 items.
Scrape page HANZ The Beautiful Boutique Hotel success!
Number of item scraped: 16 items.
Scrape page Ibis Saigon Airport success!
Number of item scraped: 31 items.
Scrape page LANGHAM Boutiqu Hotel success!
Number of item scraped: 36 items.
Scrape page Goldview Apartment Luxury success!
Number of item scraped: 37 items.
Scrape page Nicecy Saigon Hotel success!
Number of item scraped: 49 items.
Scrape page Nicecy NganHa Hotel success!
Number of item scraped: 62 items.
Scrape page Nicecy Hotel - Bui Thi Xuan Street success!
Number of item scraped: 72 items.
Scrape page Hoa De Nhat Hotel- Sân Bay Tân Sơn Nhất success!
Number of item scraped: 78 items.
Scrape page Ruby Star Hotel- Near Bến Thành Market success!
N

## Convert raw value to number

In [79]:
import re 

def to_number(value):
    if isinstance(value, str):
        try: 
            value = re.sub(r'[^\d.]', '', value.replace(',', ''))
            return float(value)
        except:
            print("error converting {} to string".format(value))
    return value  # If already a number, return as is

for room in rooms:
    room['room_area'] = to_number(room['room_area'])
    room['discount_rate'] = to_number(room['discount_rate'])
    room['root_price'] = to_number(room['root_price'])
    room['price'] = to_number(room['price'])

print("total item: ", len(rooms))


error converting  to string
error converting  to string
error converting  to string
error converting  to string
error converting  to string
error converting  to string
error converting  to string
error converting  to string
error converting  to string
error converting  to string
error converting  to string
error converting  to string
error converting  to string
error converting  to string
error converting  to string
error converting  to string
error converting  to string
error converting  to string
error converting  to string
error converting  to string
error converting  to string
error converting  to string
error converting  to string
error converting  to string
error converting  to string
error converting  to string
total item:  1178


## Save data to csv files

In [81]:
import csv

def save_to_csv(data, filename='room_price.csv'):
    # Define fieldnames based on the keys of the first dictionary in the list
    fieldnames = data[0].keys() if data else []

    # Write data to CSV file
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        # Write header
        writer.writeheader()
        
        # Write rows
        for row in data:
            writer.writerow(row)


# save cleaned data to csv
save_to_csv(rooms)


## Analyze data



In [83]:
import pandas as pd
df = pd.DataFrame(rooms)

df

,hotel_name,hotel_star,room_name,room_area,bed_detail,available_guests,root_price,discount_rate,price,description
0,Ruby Saigon Hotel Le Thanh Ton,3,Superior Double or Twin Room,20.0,"2 twin beds,1 queen bed",2,6000000.0,85.0,900000.0,None
1,Ruby Saigon Hotel Le Thanh Ton,3,Deluxe Double or Twin Room,20.0,"1 queen bed,2 twin beds",2,6300000.0,85.0,945000.0,None
2,Ruby Saigon Hotel Le Thanh Ton,3,Deluxe Triple Room,28.0,"1 twin bed,1 queen bed",3,6600000.0,85.0,990000.0,None
3,Ruby Saigon Hotel Le Thanh Ton,3,Suite with City View,38.0,,3,7600000.0,85.0,1140000.0,None
4,Elysium CBD Rivergate Apartment - Gym & Pool -...,0,Deluxe King Studio,35.0,1 king bed,2,450000.0,20.0,360000.0,None
...,...,...,...,...,...,...,...,...,...,...
1173,Apartments near Tân Sân Nhất Airpot,4,Two-Bedroom Apartment,70.0,,4,2090000.0,22.0,1630200.0,None
1174,Apartments near Tân Sân Nhất Airpot,4,Three-Bedroom Apartment,105.0,,1,2999000.0,22.0,2339220.0,None
1175,A&EM Phan Boi Chau,3,Senior Deluxe Double Room,22.0,1 queen bed,2,2250000.0,46.0,1215000.0,None
1176,A&EM Phan Boi Chau,3,Executive Double Room,28.0,1 queen bed,2,2500000.0,46.0,1350000.0,None


## Describe data

In [84]:
# Generate descriptive statistics
description = df.describe(include='all')

# Print the descriptive statistics
print(description)


                   hotel_name   hotel_star             room_name   room_area  \
count                    1178  1178.000000                   769  716.000000   
unique                    174          NaN                   278         NaN   
top     Sunrise Central Hotel          NaN  Superior Double Room         NaN   
freq                       33          NaN                    38         NaN   
mean                      NaN     2.681664                   NaN   40.706704   
std                       NaN     1.494147                   NaN   30.489856   
min                       NaN     0.000000                   NaN   15.000000   
25%                       NaN     2.000000                   NaN   22.750000   
50%                       NaN     3.000000                   NaN   30.000000   
75%                       NaN     4.000000                   NaN   42.000000   
max                       NaN     5.000000                   NaN  200.000000   

       bed_detail  available_guests    